<a href="https://colab.research.google.com/github/the-steelix-flame/FACERECO/blob/main/Facerecomain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=7ba641ceb79f0293ddcc521753ad8eda6733261ce78d67b89e25140cf19f6495
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [2]:
import cv2
import face_recognition

def preprocess_image(image_path):
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)
    if face_locations:
        top, right, bottom, left = face_locations[0]
        face_image = image[top:bottom, left:right]
        return cv2.resize(face_image, (160, 160))
    return None

In [3]:
import face_recognition
import pickle

def generate_encodings(images_dir):
    encodings = {}
    for image_path in os.listdir(images_dir):
        employee_id = os.path.splitext(image_path)[0]
        image = preprocess_image(os.path.join(images_dir, image_path))
        if image is not None:
            encodings[employee_id] = face_recognition.face_encodings(image)[0]
    with open('data/encodings.pkl', 'wb') as f:
        pickle.dump(encodings, f)

In [4]:
import cv2
import face_recognition
import pickle

def recognize_faces():
    with open('data/encodings.pkl', 'rb') as f:
        encodings = pickle.load(f)
    video_capture = cv2.VideoCapture(0)
    while True:
        ret, frame = video_capture.read()
        face_locations = face_recognition.face_locations(frame)
        face_encodings = face_recognition.face_encodings(frame, face_locations)
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(list(encodings.values()), face_encoding)
            if True in matches:
                employee_id = list(encodings.keys())[matches.index(True)]
                print(f"Employee {employee_id} recognized")
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    video_capture.release()
    cv2.destroyAllWindows()

In [5]:
import sqlite3

def create_database():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS employees
                  (id TEXT PRIMARY KEY, name TEXT, encoding BLOB)''')
    c.execute('''CREATE TABLE IF NOT EXISTS attendance
                  (id TEXT, entry_time DATETIME)''')
    conn.commit()
    conn.close()

In [6]:
def add_employee(employee_id, name, encoding):
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("INSERT INTO employees VALUES (?, ?, ?)", (employee_id, name, encoding))
    conn.commit()
    conn.close()

In [7]:
import datetime

def log_entry(employee_id):
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("INSERT INTO attendance VALUES (?, ?)", (employee_id, datetime.datetime.now()))
    conn.commit()
    conn.close()

In [ ]:
from flask import Flask, render_template
import sqlite3

app = Flask(__name__)

@app.route('/')
def index():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT * FROM attendance")
    records = c.fetchall()
    conn.close()
    return render_template('index.html', records=records)

if __name__ == '__main__':
    app.run(debug=True)